<a href="https://colab.research.google.com/github/rmccormick314/GFSAD/blob/main/preprocessing/Stacked_Image_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Set Up

## a. Requirements

In [1]:
repo_url = "https://raw.githubusercontent.com/GoogleCloudPlatform/python-docs-samples/main/people-and-planet-ai/land-cover-classification"

!wget --quiet {repo_url}/requirements.txt

!pip install --quiet --upgrade pip
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

## b. Set Project Info
Authenticates the user to Earth Engine and sets GCP variables.

In [4]:
from __future__ import annotations

import os
from google.colab import auth

auth.authenticate_user()

# Please fill in these values.
project = "gfsad-446404"  # @param {type:"string"}
bucket = "lgcip30"  # @param {type:"string"}
location = "us-west1"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith(
    "gs://"
), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

Updated property [core/project].


## c. Clone GitHub Code

In [2]:
# Now let's get the code from GitHub and navigate to the sample.
!git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
%cd python-docs-samples/people-and-planet-ai/land-cover-classification

Cloning into 'python-docs-samples'...
remote: Enumerating objects: 118950, done.
remote: Counting objects: 100% (6982/6982), done.
remote: Compressing objects: 100% (1263/1263), done.
remote: Total 118950 (delta 6443), reused 5721 (delta 5719), pack-reused 111968 (from 3)
Receiving objects: 100% (118950/118950), 242.97 MiB | 27.67 MiB/s, done.
Resolving deltas: 100% (71875/71875), done.
Updating files: 100% (5500/5500), done.
/content/python-docs-samples/people-and-planet-ai/land-cover-classification


## d. Initialize Earth Engine

In [5]:
import ee
import google.auth

credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

# II. Create Dataset

## a. Set Parameters

In [ ]:
YEAR = "2020" # @param {"type":"string"}
GAEZ = "8" # @param {"type":"string"}

## b. Visualize CDL

In [9]:
import ee
import folium

CLASSIFICATIONS = {
    "Perennial" : "1E90FF",
    "Fodder"    : "00FF00",
    "Dbl. Crop" : "FFD700",
    "Annual"    : "FF4500",
    "Fallow"    : "bfbf77"
}

image = (
    ee.Image("USDA/NASS/CDL/" + YEAR)
    .select("cropland")
    .remap(
        [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 81, 82, 83, 87, 88, 92, 111, 112, 121, 122, 123, 124, 131, 141, 142, 143, 152, 176, 190, 195, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 254],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 5, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 4, 1, 4, 4, 1, 1, 1, 4, 4, 1, 4, 1, 1, 4, 1, 1, 4, 1, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 4, 4, 4, 4, 4, 4, 4, 1, 3]
    )
    .rename("label")
)

vis_params = {
    "max": len(CLASSIFICATIONS),
    "palette": list(CLASSIFICATIONS.values()),
    "bands": ["label"],
}
folium.Map(
    location=(38, -106),
    zoom_start=5,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

## c. Build Image Stack

In [71]:
import logging
import datetime
import ee

states = ee.FeatureCollection("TIGER/2018/States")
S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
GAEZ = ee.FeatureCollection("users/rmccormick314/gfsad30-raez-74zones")
GCEP1k = ee.Image("USGS/GFSAD1000_V1")
LGRIP30 = ee.Image("projects/usgs-gee-wgscflag/assets/LGRIP30USA2020/LGRIP30USA2020-smoothed70m")
ESA = ee.ImageCollection("ESA/WorldCover/v200")

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# Create a handler (e.g., to output to console)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

# Set the year and region
YEAR_INT = int(YEAR)
START_DATE = ee.Date( YEAR+"-01-01" )
END_DATE = ee.Date( YEAR+"-12-31" )

# Define the region of interest
logger.info("Defining region of interest")
region = ee.FeatureCollection('users/rmccormick314/gfsad30-raez-74zones')\
    .filter(ee.Filter.eq("Zone", "8"))\
    .geometry()

# Validate the geometry
if region == None:
    raise ValueError("The defined region is empty. Please check the filter criteria.")

# Load and prepare the GCEP data
logger.info("Preparing GCEP data")
GCEP = LGRIP30.clip(region)
GCEP = GCEP.updateMask(GCEP.neq(0))

# Load and process the Cropland Data Layer (CDL)
logger.info("Preparing CDL data")
CDL = ee.Image("USDA/NASS/CDL/"+YEAR).select("cropland").clip(region)
attribute_codes = [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 81, 82, 83, 87, 88, 92, 111, 112, 121, 122, 123, 124, 131, 141, 142, 143, 152, 176, 190, 195, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 254]
reclass_codes = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 5, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 4, 1, 4, 4, 1, 1, 1, 4, 4, 1, 4, 1, 1, 4, 1, 1, 4, 1, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 4, 4, 4, 4, 4, 4, 4, 1, 3]
CDL = CDL.remap(attribute_codes, reclass_codes).rename(["CropType"]).updateMask(GCEP)


geometry = ee.Geometry.Polygon([
    [[-106.0, 37.0], [-106.0, 38.0], [-105.0, 38.0], [-105.0, 37.0], [-106.0, 37.0]]
])

# Load and process Sentinel-2 data
logger.info("Preparing Sentinel-2 data")
S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
    .filterDate(START_DATE, END_DATE)\
    .filterBounds(region)\
    .map(lambda img: img.clip(region).updateMask(GCEP))


In [72]:
# Calculate NDVI for each image
logger.info("Calculating NDVI")
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

S2 = S2.map(calculate_ndvi).select('NDVI')

# Temporal stacking
logger.info("Creating temporal stacks")
def temporal_stack(dates):
    def stack_period(date_range):
        start, end = date_range
        period_images = S2.filterDate(start, end)
        mosaic = period_images.max() if period_images.size().getInfo() > 0 else ee.Image(0).rename('NDVI')
        return mosaic

    return ee.ImageCollection([stack_period(d) for d in dates])

date_ranges = [
    ('2020-03-01', '2020-03-31'), ('2020-04-01', '2020-04-30'),
    ('2020-05-01', '2020-05-31'), ('2020-06-01', '2020-06-30'),
]
stacked_images = temporal_stack(date_ranges)
stacked_image = stacked_images.toBands().clip(region)

# Save or use image
logger.info("Exporting image to Google Cloud Storage")
output_bucket = bucket
output_project = project

def export_to_bucket(image, bucket, region):
    task = ee.batch.Export.image.toCloudStorage(
        image=image,
        description='stacked_image_export',
        bucket=bucket,
        fileNamePrefix='stacked_image',
        region=region,  # Pass the GeoJSON bounding box
        scale=30,
        maxPixels=1e13
    )
    task.start()
    logger.info("Export task started: %s", task.id)

export_to_bucket(stacked_image, output_bucket, region)
logger.info("Script finished.")
